<a href="https://colab.research.google.com/github/eunicean/Proyecto3-Paralela/blob/main/proyecto3_paralela.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/chuy-zip/PROYECTO2_DS.git

Cloning into 'PROYECTO2_DS'...
remote: Enumerating objects: 4328, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 4328 (delta 33), reused 38 (delta 13), pack-reused 4264 (from 2)
Receiving objects: 100% (4328/4328), 63.39 MiB | 15.89 MiB/s, done.
Resolving deltas: 100% (59/59), done.
Updating files: 100% (4208/4208), done.


In [ ]:
print("creacion de jupyter")